# Analiza danych Facebooka w latach 2008-2022

## Cel projektu
Celem projektu jest analiza i modelowanie kluczowych wskaźników biznesowych Facebooka, w tym:
1. Liczby użytkowników w czasie
2. Zależności między przychodami a liczbą użytkowników
3. Relacji między kosztami a zatrudnieniem
4. Wielowymiarowego modelu kosztów

## Struktura analizy
1. Przygotowanie i eksploracja danych (2008-2017)
2. Budowa modeli predykcyjnych
3. Walidacja modeli na danych testowych (2018-2022)
4. Analiza błędów i ocena jakości modeli

## Wykorzystane metody
- Regresja liniowa
- Transformacja logarytmiczna danych
- Analiza szeregów czasowych
- Modelowanie wielowymiarowe

## Import paczek potrzebnych do laboratorium

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import mean_squared_error
# Obliczanie przedziału ufności
from sklearn.metrics import mean_squared_error

## Liczba uzytkownikow w kolejnych kwartalach (Facebook)

In [ ]:
user_count_q = {
    'Q':
        ['Q3 08', 'Q1 09', 'Q2 09', 'Q3 09', 'Q4 09', 'Q1 10',
         'Q2 10', 'Q3 10', 'Q4 10', 'Q1 11', 'Q2 11', 'Q3 11',
         'Q4 11', 'Q1 12', 'Q2 12', 'Q3 12', 'Q4 12', 'Q1 13',
         'Q2 13', 'Q3 13', 'Q4 13', 'Q1 14', 'Q2 14', 'Q3 14',
         'Q4 14', 'Q1 15', 'Q2 15', 'Q3 15', 'Q4 15', 'Q1 16',
         'Q2 16', 'Q3 16', 'Q4 16', 'Q1 17', 'Q2 17', 'Q3 17', 'Q4 17'],
    'User count':
        [100, 197, 242, 305, 360, 431, 482, 550, 608, 680, 739, 800,
         845, 901, 955, 1007, 1056, 1110, 1155, 1189, 1228, 1276, 1317,
         1350, 1393, 1441, 1490, 1545, 1591, 1654, 1712, 1788, 1860, 1936,
         2006, 2072, 2129]
}

## Liczba uzytkownikow w kolejnych kwartalach (Facebook) - dane z lat 2008-2017 - usuwanie roku 2007 gdyz nie ma danych dla poprzedniej tabeli z tego roku


In [ ]:
fb_stats_y_old = {
    'Y': [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
    'Income': [153, 272, 777, 1974, 3711, 5089, 7872, 12466, 17928, 27638, 40653],
    'Profit': [-138, -56, 229, 606, 1000, 53, 1500, 2940, 3688, 10217, 15934],
    'Employment': [450, 850, 1218, 2127, 3200, 4619, 6337, 9199, 12691, 17048, 25105]
}

# Poprawione dane z 2008-2017
fb_stats_y = {
    'Y': [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
    'Income': [272, 777, 1974, 3711, 5089, 7872, 12466, 17928, 27638, 40653],
    'Profit': [-56, 229, 606, 1000, 53, 1500, 2940, 3688, 10217, 15934],
    'Employment': [850, 1218, 2127, 3200, 4619, 6337, 9199, 12691, 17048, 25105]
}

## Tworzenie DataFrame z danymi 

In [ ]:
user_count_df = pd.DataFrame(data=user_count_q)
fb_stats_y_df = pd.DataFrame(data=fb_stats_y)
print(fb_stats_y_df)
print(user_count_df)

In [ ]:
def convert_quarter_to_year(quarter) -> int:
    year_str = quarter.split(' ')[1]
    year_int = 2000 + int(year_str)
    return year_int


In [ ]:
user_count_y_df = user_count_df.copy()
user_count_y_df['Q'] = user_count_df['Q'].map(convert_quarter_to_year)
# Alternatywa byloby zaladowanie danych z Q4 jako max tez dobrze
user_count_y_df = user_count_y_df
user_count_y_df = user_count_y_df.groupby('Q', as_index=False).max()
user_count_y_df.rename(columns={'Q': 'Y'}, inplace=True)
print(user_count_y_df)

In [ ]:
fb_stats_y_df = pd.DataFrame(data=fb_stats_y)
# Calculate costs
fb_stats_y_df['Costs'] = fb_stats_y_df['Income'] - fb_stats_y_df['Profit']
# fb_stats_y_df.drop([0], inplace=True)
fb_stats_y_df.reset_index(drop=True, inplace=True)
fb_stats_y_df.insert(3, 'User count', user_count_y_df['User count'])
print(fb_stats_y_df)


## Tworzenie wykresu iloczynu kartezjanskiego w celu wyzanczeniach zaleznosci miedzy danymi

In [ ]:
pd.plotting.scatter_matrix(fb_stats_y_df, figsize=(10, 10))

## Dane dla przyszlych lat (2018-2022) potrzbne do weryfikowania predyckji modelu

In [ ]:
user_count_q_test = {
    'Q':
        ['Q1 18', 'Q2 18', 'Q3 18', 'Q4 18',
         'Q1 19', 'Q2 19', 'Q3 19', 'Q4 19',
         'Q1 20', 'Q2 20', 'Q3 20', 'Q4 20',
         'Q1 21', 'Q2 21', 'Q3 21', 'Q4 21',
         'Q1 22', 'Q2 22', 'Q3 22', 'Q4 22'],
    'User count':
        [2196, 2234, 2271, 2320, 2375, 2414, 2449, 2498, 2603, 2701, 2740, 2797,
         2853, 2895, 2910, 2912, 2936, 2934, 2958, 2963]
}

fb_stats_y_test = {
    'Y': [2018, 2019, 2020, 2021, 2022],
    'Income': [55838, 70697, 85965, 117929, 116609],
    'Profit': [22112, 18485, 29146, 39370, 23200],
    'Costs': [33726, 52212, 56819, 78559, 93409],
    'Employment': [35587, 44942, 58604, 71970, 87314],
    'User count': [2320, 2498, 2797, 2912, 2963]
}

In [ ]:
user_count_test_df = pd.DataFrame(data=user_count_q_test)
fb_stats_y_test_df = pd.DataFrame(data=fb_stats_y_test)
print(user_count_test_df)
print(fb_stats_y_test_df)

## Funkcja do obliczania statystyki modelu

In [ ]:
def calculate_model_stats(y_hat, y_true, X) -> dict:
    # Dodajemy kolumne jedynek dla wyrazu wolnego 
    X = np.insert(X, 0, 1, axis=1)

    residuals = y_true - y_hat
    # Obliczamy sume kwadratow reszt(RSS - Residual Sum of Squares)
    # RSS = e^T * e, gdzie e to wektor reszt
    residuals_sum_squared = residuals.T @ residuals

    # Obliczamy estymator wariancji resztowej (σ²)
    # σ² = RSS/(n-k), gdzie:
    # n = liczba obserwacji (y_hat.shape[0])
    # k = liczba parametrów modelu (X.shape[1])
    standard_variance = residuals_sum_squared[0, 0] / (y_hat.shape[0] - X.shape[1])

    # Var(β̂) = σ² × (X^T × X)^(-1)
    model_coefficients_covariance = standard_variance * np.linalg.inv(X.T @ X)  # Liczymy macierz kowariancji wspl. modelu
    stats = dict()

    # Obliczamy błędy standardowe dla każdego współczynnika
    # SE(β̂ᵢ) = √Var(β̂ᵢ)
    for number in range(model_coefficients_covariance.shape[0]):
        stats['Standard error a' + str(number)] = np.sqrt(model_coefficients_covariance[number, number])

    # Na koniec blad standardowy modelu
    stats['Standard error e'] = np.sqrt(standard_variance)
    return stats



## Funkcje pomocnicze dla oceny predykcji modelu MSE, RMSE MAE i MAPE

In [ ]:
def calculate_prediction_metrics(y_true, y_pred):
    # Parametry funkcji: y_true - rzeczywiste wartości, y_pred - przewidywane wartości
    # Konwersja do numpy array dla pewności
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    
    # Mean Squared Error (MSE)
    mse = np.mean((y_true - y_pred) ** 2)
    
    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    
    # Mean Absolute Error (MAE)
    mae = np.mean(np.abs(y_true - y_pred))
    
    # Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    # R-squared (R²)
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    
    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'R2': r2
    }

def print_metrics(metrics, model_name="Model"):
    def get_error_rating(metric_name, value):
        if metric_name == 'MAPE':
            if value < 10: return "Bardzo dobry"
            elif value < 20: return "Dobry"
            elif value < 30: return "Umiarkowany"
            else: return "Słaby"
        elif metric_name == 'R2':
            if value > 0.9: return "Bardzo dobry"
            elif value > 0.7: return "Dobry"
            elif value > 0.5: return "Umiarkowany"
            else: return "Słaby"
        return ""  # dla pozostałych metryk nie wyświetlamy oceny

    def get_unit(metric_name):
        units = {
            'MSE': '(jednostka²)',
            'RMSE': '(jednostka)',
            'MAE': '(jednostka)',
            'MAPE': '%',
            'R2': '(0-1)'
        }
        return units.get(metric_name, '')

    print(f"\nMetryki oceny dla {model_name}:")
    print("-" * 60)
    print(f"{'Metryka':<10} {'Wartość':<15} {'Jednostka':<15} {'Ocena':<15}")
    print("-" * 60)
    
    for metric, value in metrics.items():
        unit = get_unit(metric)
        rating = get_error_rating(metric, value)
        print(f"{metric:<10} {value:>15.4f} {unit:<15} {rating:<15}")
        
        
def print_metrics_with_specific_units(metrics, model_name="Model", unit_name=""):
    def get_error_rating(metric_name, value):
        if metric_name == 'MAPE':
            if value < 10: return "Bardzo dobry"
            elif value < 20: return "Dobry"
            elif value < 30: return "Umiarkowany"
            else: return "Słaby"
        elif metric_name == 'R2':
            if value > 0.9: return "Bardzo dobry"
            elif value > 0.7: return "Dobry"
            elif value > 0.5: return "Umiarkowany"
            else: return "Słaby"
        return ""

    def get_unit(metric_name, unit_name):
        units = {
            'MSE': f"({unit_name}²)",
            'RMSE': f"({unit_name})",
            'MAE': f"({unit_name})",
            'MAPE': '%',
            'R2': '(0-1)'
        }
        return units.get(metric_name, '')

    print(f"\nMetryki oceny dla {model_name}:")
    print("-" * 70)
    print(f"{'Metryka':<10} {'Wartość':<15} {'Jednostka':<20} {'Ocena':<15}")
    print("-" * 70)
    
    for metric, value in metrics.items():
        unit = get_unit(metric, unit_name)
        rating = get_error_rating(metric, value)
        print(f"{metric:<10} {value:>15.4f} {unit:<20} {rating:<15}")

## Funkcje pomocnicze dla szeregow czasowych z uzyciem kwartalow

In [ ]:
def transform_quarter_to_number(string) -> int:
    if ' ' not in string:
        raise ValueError(f"Invalid format for quarter: {string}")
    quarter, year = string.split(' ')
    quarters = {'Q1': 0, 'Q2': 1, 'Q3': 2, 'Q4': 3}
    year = int(year) - 8
    return year * 4 + quarters[quarter]


def transform_number_to_quarter(number) -> str:
    year = (number // 4) + 8
    quarters = {0: 'Q1', 1: 'Q2', 2: 'Q3', 3: 'Q4'}
    quarter = quarters[number % 4]
    quarter_string = quarter + ' {:02d}'.format(year)
    return quarters[quarter] + ' ' + str(year)


In [ ]:
user_count_df['Q'] = user_count_df['Q'].astype(str).map(transform_quarter_to_number)
user_count_test_df['Q'] = user_count_test_df['Q'].astype(str).map(transform_quarter_to_number)
print(user_count_df)
print(user_count_test_df)

## Trenowanie modelu szeregu czasowego 

In [ ]:
R_train = user_count_df['Q'].to_numpy().reshape(-1, 1)
U_train = user_count_df['User count'].to_numpy().reshape(-1, 1)
R_test = user_count_test_df['Q'].to_numpy().reshape(-1, 1)
U_test = user_count_test_df['User count'].to_numpy().reshape(-1, 1)

linear_regression = LinearRegression()
linear_regression.fit(R_train, U_train)

# Ocena modelu na danych testowych
train_metrics_0 = calculate_prediction_metrics(U_train, linear_regression.predict(R_train))
test_metrics_0 = calculate_prediction_metrics(U_test, linear_regression.predict(R_test))

print_metrics(train_metrics_0, "Model szeregu czasowego (dane treningowe)")
print_metrics(test_metrics_0, "Model szeregu czasowego (dane testowe)")

print(linear_regression)

print(f'User count R^2: {linear_regression.score(R_train, U_train):}')
print(f"Model: U = {linear_regression.intercept_[0]:} + {linear_regression.coef_[0][0]:} * R")

## Errors for out model

In [ ]:
errors = calculate_model_stats(linear_regression.predict(R_train), U_train, R_train)
# print(errors)

print("Errors for model:")
for key, value in errors.items():
    print(f"{key}: {float(value):.4f}")

## Wizualizacja modelu szeregu czasowego

In [ ]:
# Funckja zmieniona w celu lepszej czytelnosci
def transform_number_to_quarter_custom(number) -> str:
    quarters = {0: 'Q1', 1: 'Q2', 2: 'Q3', 3: 'Q4'}
    year = 2008 + (number // 4)
    quarter = quarters[number % 4]
    return f'{quarter} {year}'


# Przygotowanie danych
train_predictions = linear_regression.predict(R_train)
test_predictions = linear_regression.predict(R_test)

# Obliczanie przedziału ufności
mse = mean_squared_error(U_train, train_predictions)
std_dev = np.sqrt(mse)
confidence_interval = 1.96 * std_dev  # 95% przedział ufności

# Tworzenie wykresu z większym rozmiarem i lepszymi proporcjami
plt.figure(figsize=(16, 9))


# Konfiguracja stylu
plt.style.use('seaborn-v0_8-deep')
colors = {
    'train_dots': '#1f77b4',
    'train_line': '#ff0000',
    'test_dots': '#2ca02c',
    'test_line': '#ff7f0e',
    'confidence': '#fff59d'
}

# Dane treningowe
plt.scatter(R_train, U_train, color=colors['train_dots'], alpha=0.7, s=60,
            label='Rzeczywista liczba użytkowników (dane uczące)',
            edgecolor='white', linewidth=1)
plt.plot(R_train, train_predictions, color=colors['train_line'], linewidth=2.5,
         label='Prognozowana liczba użytkowników (dane uczące)')

# Dane testowe
plt.scatter(R_test, U_test, color=colors['test_dots'], alpha=0.7, s=60,
            label='Rzeczywista liczba użytkowników (dane testowe)',
            edgecolor='white', linewidth=1)
plt.plot(R_test, test_predictions, color=colors['test_line'], linewidth=2.5,
         label='Prognozowana liczba użytkowników (dane testowe)')

# Przedział ufności z lepszą przezroczystością i kolorem
all_quarters = np.concatenate([R_train.flatten(), R_test.flatten()])
all_predictions = np.concatenate([train_predictions.flatten(), test_predictions.flatten()])
plt.fill_between(
    all_quarters,
    all_predictions - confidence_interval,
    all_predictions + confidence_interval,
    color=colors['confidence'],
    alpha=0.3,
    label='95% przedział ufności'
)

# Formatowanie osi i etykiet
plt.xlabel('Kwartał', fontsize=12, fontweight='bold')
plt.ylabel('Liczba użytkowników [mln]', fontsize=12, fontweight='bold')
plt.title('Prognoza liczby użytkowników Facebooka w latach 2008-2022',
          fontsize=16, pad=20, fontweight='bold')

# Generowanie etykiet osi X używając zdefiniowanych funkcji
x_ticks = range(0, 60)
x_labels = [transform_number_to_quarter_custom(q) for q in x_ticks]
plt.xticks(x_ticks, x_labels, rotation=45, ha='right')

# Dodanie siatki z lepszą widocznością
plt.grid(True, linestyle='--', alpha=0.3)

# Lepsze umiejscowienie legendy i dodanie ramki
legend = plt.legend(bbox_to_anchor=(1.15, 1), loc='upper left',
                    frameon=True, fancybox=True, shadow=True)
legend.get_frame().set_alpha(0.9)

# Dodanie informacji o R² w lepszym formacie
r2_score = linear_regression.score(R_train, U_train)
stats_text = f'R² (test) = {r2_score:.4f}\n'
stats_text += f'RMSE = {np.sqrt(mse):.2f} mln'

plt.text(0.02, 0.95, stats_text,
         transform=plt.gca().transAxes,
         bbox=dict(facecolor='white',
                   edgecolor='gray',
                   alpha=0.8,
                   boxstyle='round,pad=0.5'),
         fontsize=10)

# Dodanie adnotacji dla ważnych punktów
max_users_idx = np.argmax(U_test)
plt.annotate(f'Maksimum: {U_test[max_users_idx][0]:.0f} mln',
             xy=(R_test[max_users_idx], U_test[max_users_idx]),
             xytext=(10, 10), textcoords='offset points',
             bbox=dict(facecolor='white', edgecolor='gray', alpha=0.7),
             arrowprops=dict(arrowstyle='->'))

# Dostosowanie układu i marginesów 
plt.tight_layout()

# Pokazanie wykresu
plt.show()

## Wykres income w zaleznosci od user_count

In [ ]:
# Przygotowanie danych treningowych (2008-2017)
X_train = fb_stats_y_df['User count'].to_numpy().reshape(-1, 1)  # zmienna niezależna
y_train = fb_stats_y_df['Income'].to_numpy().reshape(-1, 1)      # zmienna zależna

# Przygotowanie danych testowych (2018-2022)
X_test = fb_stats_y_test_df['User count'].to_numpy().reshape(-1, 1)
y_test = fb_stats_y_test_df['Income'].to_numpy().reshape(-1, 1)

# Trenowanie modelu
linear_regression_income = LinearRegression()
linear_regression_income.fit(X_train, y_train)  # X_train jako pierwsza zmienna

In [ ]:
# Statystyki modelu: wspolczynnik determinacji R^2
print(f'Income R^2: {linear_regression_income.score(X_train, y_train):}')

# Równanie modelu ma postać: I = wyraz wolny + współczynnik * U
# - 'intercept_' to wyraz wolny (przecięcie z osią y) linii regresji, reprezentujący przewidywaną wartość I, gdy U wynosi 0.
# - 'coef_' to nachylenie linii regresji, wskazujące, o ile zmienia się I przy jednostkowej zmianie U.
print(
    f'Model: I {linear_regression_income.intercept_[0]:} + {linear_regression_income.coef_[0][0]:} * U')  # Model rowna sie wspolczynnikowi nachylenia + wyrazowi wolnemu

## Błędy modelu

In [ ]:
# Przeliczanie błędów modelu
errors = calculate_model_stats(linear_regression_income.predict(X_train), y_train, X_train)
# print(errors)

print("Errors for model:")
for key, value in errors.items():
    print(f"{key}: {float(value):.4f}")

In [ ]:
# Przewidywania dla danych treningowych
train_predictions = linear_regression_income.predict(X_train)
print("Przewidywania dla danych treningowych:")
print(train_predictions)
print("\n" + "=" * 30 + "\n")  # Separator dla lepszej czytelności

# Przewidywania dla danych testowych
test_predictions = linear_regression_income.predict(X_test)
print("Przewidywania dla danych testowych:")
print(test_predictions)

## Zmiana koncepcji testowanie modelu dla usunietych 3 pierwszych danych dla mniej problematycznej prostej

In [ ]:
# Przygotowanie danych treningowych i testowych
# U_train_1 i P_train_1 zawierają wszystkie dane, podczas gdy U_train_2 i P_train_2 pomijają pierwsze trzy punkty
U_train_1 = fb_stats_y_df['User count'].to_numpy().reshape(-1, 1)
P_train_1 = fb_stats_y_df['Income'].to_numpy().reshape(-1, 1)
P_train_log_1 = np.log10(P_train_1)  # Logarytmowanie przychodów dla lepszej liniowości

# Narysuj wykresy dla obu zestawów danych
plt.figure(figsize=(10, 6))
plt.grid(True, linestyle='--', alpha=0.3)
plt.scatter(U_train_1, P_train_log_1, label='Przychód rzeczywisty')
plt.xlabel('Liczba użytkowników [mln]')
plt.ylabel('Przychód [mln]')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.grid(True, linestyle='--', alpha=0.3)
plt.scatter(U_train_1, P_train_1, label='Przychód rzeczywisty')
plt.xlabel('Liczba użytkowników [mln]')
plt.ylabel('Przychód [mln]')
plt.legend()
plt.show()


## Logarytmizacja danych oraz testowanie modelu liniowej regresji dla zadanych punktow

In [ ]:
# Przygotowanie danych treningowych i testowych
U_train_1 = fb_stats_y_df['User count'].to_numpy().reshape(-1, 1)
P_train_1 = fb_stats_y_df['Income'].to_numpy().reshape(-1, 1)
P_train_log_1 = np.log10(P_train_1)  # Logarytmowanie przychodów dla lepszej liniowości

# Ile punktów chcemy usunąć
points_to_remove = 3

# Wydzielenie usuniętych punktów i pozostałych
U_removed = U_train_1[:points_to_remove]  # Pierwsze 3 punkty
P_removed_log = P_train_log_1[:points_to_remove]  # Logarytmy pierwszych 3 punktów
U_kept = U_train_1[points_to_remove:]  # Pozostałe punkty
P_kept_log = P_train_log_1[points_to_remove:]  # Logarytmy pozostałych punktów

# Tworzenie i trenowanie modeli regresji liniowej
l1 = LinearRegression()
l1.fit(U_train_1, P_train_log_1)  # Model uwzględniający wszystkie punkty

l2 = LinearRegression()
l2.fit(U_kept, P_kept_log)  # Model pomijający pierwsze trzy punkty

# Pobieranie współczynników i wyrazów wolnych z obu modeli
a11 = l1.coef_[0]
a01 = l1.intercept_
a12 = l2.coef_[0]
a02 = l2.intercept_

# Przewidywanie przychodów na podstawie obu modeli
P_pred_1 = a11 * U_train_1 + a01
P_pred_2 = a12 * U_train_1 + a02

# Wizualizacja danych i modeli
plt.clf()
plt.grid(True, linestyle='--', alpha=0.3)

# Punkty zachowane na zielono
plt.scatter(U_kept, P_kept_log, color='green', label='Punkty zachowane')
# Punkty usunięte na czerwono
plt.scatter(U_removed, P_removed_log, color='red', label='Punkty usunięte')

plt.plot(U_train_1, P_pred_1, label='Linia uwzględniająca wszystkie punkty')
plt.plot(U_train_1, P_pred_2, label='Linia bez 3 pierwszych punktów')

plt.xlabel('Liczba użytkowników [mln]')
plt.ylabel('Przychód [log10(mln)]')
plt.legend()
plt.show()

print(
    f'Model: I {l2.intercept_[0]:} + {l2.coef_[0][0]:} * U')  # Model rowna sie wspolczynnikowi nachylenia + wyrazowi wolnemu

## Bledy modelu

In [ ]:
# Przygotowanie danych do ostatecznego modelu
U_train = fb_stats_y_df['User count'].to_numpy()[3:].reshape(-1, 1)
P_train = fb_stats_y_df['Income'].to_numpy()[3:].reshape(-1, 1)
P_train_log = np.log10(P_train)

U_test = fb_stats_y_test_df['User count'].to_numpy().reshape(-1, 1)
P_test = fb_stats_y_test_df['Income'].to_numpy().reshape(-1, 1)
P_test_log = np.log10(P_test)

# Trenowanie ostatecznego modelu regresji liniowej
lin_reg_income = LinearRegression()
lin_reg_income.fit(U_train, P_train_log)

# Ocena modelu
print(f'Income R^2: {lin_reg_income.score(U_train, P_train_log)}')
errors = calculate_model_stats(lin_reg_income.predict(U_train), P_train_log, U_train)
print("Errors for model:")
for key, value in errors.items():
    print(f"{key}: {float(value):.4f}")
    
# print(errors) # For the a1 bc -05e
P_error = errors['Standard error e']

# Ocena modelu na danych zlogarytmowanych
train_metrics_1 = calculate_prediction_metrics(P_train_log, lin_reg_income.predict(U_train))
test_metrics_1 = calculate_prediction_metrics(P_test_log, lin_reg_income.predict(U_test))

print_metrics_with_specific_units(train_metrics_1, "Model przychodów zlogarytmowanych (dane treningowe)", "log10(mln $)")
print_metrics_with_specific_units(test_metrics_1, "Model przychodów zlogarytmowanych (dane testowe)", "log10(mln $)")

## Wizualizacja modelu

In [ ]:
# Pobieranie współczynników i wyrazu wolnego z ostatecznego modelu
a1 = lin_reg_income.coef_[0]
a0 = lin_reg_income.intercept_

# Przewidywanie przychodów dla danych treningowych i testowych
P_train_pred = a1 * U_train + a0
P_test_pred = a1 * U_test + a0

# Wizualizacja wyników modelu
plt.figure(figsize=(10, 6))
# Dodanie siatki z lepszą widocznością
plt.grid(True, linestyle='--', alpha=0.3)
plt.fill_between(U_train.reshape(-1), 
                 (P_train_pred - P_error).reshape(-1), 
                 (P_train_pred + P_error).reshape(-1), 
                 color='yellow', 
                 label='Odchylenie standardowe prognozy (dane uczące)')
plt.fill_between(U_test.reshape(-1), 
                 (P_test_pred - P_error).reshape(-1), 
                 (P_test_pred + P_error).reshape(-1), 
                 color='yellow', 
                 alpha=0.5, 
                 label='Odchylenie standardowe prognozy (dane testowe)')
plt.scatter(U_train, P_train_log, label="Przychód prawdziwy (dane uczące)")
plt.plot(U_train, P_train_pred, label='Przychód przewidywany (dane uczące)', color='red')
plt.scatter(U_test, P_test_log, label="Przychód prawdziwy (dane testowe)", color='green')
plt.plot(U_test, P_test_pred, label='Przychód przewidywany (dane testowe)', color='orange')
plt.ylabel('Przychód [mln]')
plt.xlabel('Liczba użytkowników [mln]')
plt.legend()
plt.show()

## Wykres zależności Costs od Employment (Jak kształtują się koszty w zależności od zatrudnienia)

In [ ]:
#  Wykres zależności Costs od Employment (Jak kształtują się koszty w zależności od zatrudnienia)
X_train = fb_stats_y_df['Employment'].to_numpy().reshape(-1, 1)
y_train = fb_stats_y_df['Costs'].to_numpy().reshape(-1, 1)
X_test = fb_stats_y_test_df['Employment'].to_numpy().reshape(-1, 1)
y_test = fb_stats_y_test_df['Costs'].to_numpy().reshape(-1, 1)

linear_regression_cost = LinearRegression()
linear_regression_cost.fit(X_train, y_train)

print(linear_regression_cost)
print(f'Cost R^2: {linear_regression_cost.score(X_train, y_train):}')
print(f"Model: P = {linear_regression_cost.intercept_[0]:} + {linear_regression_cost.coef_[0][0]:} * X")


# Standardowe błędy:

In [ ]:
errors = calculate_model_stats(linear_regression_cost.predict(X_train), y_train, X_train)
# print(errors)

print("Errors for model:")
for key, value in errors.items():
    print(f"{key}: {float(value):.4f}")
    
train_metrics_2 = calculate_prediction_metrics(y_train, linear_regression_cost.predict(X_train))
test_metrics_2 = calculate_prediction_metrics(y_test, linear_regression_cost.predict(X_test))

print_metrics_with_specific_units(train_metrics_2, "Model kosztów vs zatrudnienie (dane treningowe)", "mln $")
print_metrics_with_specific_units(test_metrics_2, "Model kosztów vs zatrudnienie (dane testowe)", "mln $")

## Wizualizcja modelu Kosztów od Zatrudnienia:

In [ ]:
# Przygotowanie danych
train_predictions = linear_regression_cost.predict(X_train)
test_predictions = linear_regression_cost.predict(X_test)

# Obliczanie przedziału ufności
mse = mean_squared_error(y_train, train_predictions)
std_dev = np.sqrt(mse)
confidence_interval = 1.96 * std_dev  # 95% przedział ufności

# Tworzenie wykresu z większym rozmiarem i lepszymi proporcjami
plt.figure(figsize=(16, 9))

# Konfiguracja stylu
plt.style.use('seaborn-v0_8-deep')
colors = {
    'train_dots': '#1f77b4',  # Stonowany niebieski dla punktów treningowych
    'train_line': '#ff0000',  # Ciepły czerwony dla linii treningowej
    'test_dots': '#2ca02c',  # Łagodny zielony dla punktów testowych
    'test_line': '#ff7f0e',  # Jasny fiolet dla linii testowej
    'confidence': '#fff59d'  # Ciepły żółty dla przedziału ufności
}

# Dane treningowe
plt.scatter(X_train, y_train, color=colors['train_dots'], alpha=0.7, s=60,
            label='Rzeczywiste koszty (dane uczące)',
            edgecolor='white', linewidth=1)
plt.plot(X_train, train_predictions, color=colors['train_line'], linewidth=2.5,
         label='Prognozowane koszty (dane uczące)')

# Dane testowe
plt.scatter(X_test, y_test, color=colors['test_dots'], alpha=0.7, s=60,
            label='Rzeczywiste koszty (dane testowe)',
            edgecolor='white', linewidth=1)
plt.plot(X_test, test_predictions, color=colors['test_line'], linewidth=2.5,
         label='Prognozowane koszty (dane testowe)')

# Przedział ufności z lepszą przezroczystością i kolorem
all_employment = np.concatenate([X_train.flatten(), X_test.flatten()])
all_predictions = np.concatenate([train_predictions.flatten(), test_predictions.flatten()])
plt.fill_between(
    all_employment,
    all_predictions - confidence_interval,
    all_predictions + confidence_interval,
    color=colors['confidence'],
    alpha=0.3,
    label='95% przedział ufności'
)

# Formatowanie osi i etykiet
plt.xlabel('Liczba zatrudnień', fontsize=12, fontweight='bold')
plt.ylabel('Koszty [mln]', fontsize=12, fontweight='bold')
plt.title('Prognoza kosztów Facebooka w zależności od liczby zatrudnień',
          fontsize=16, pad=20, fontweight='bold')

# Dodanie siatki z lepszą widocznością
plt.grid(True, linestyle='--', alpha=0.3)

# Lepsze umiejscowienie legendy i dodanie ramki
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left',
                    frameon=True, fancybox=True, shadow=True)
legend.get_frame().set_alpha(0.9)

# Dodanie informacji o R² w lepszym formacie
r2_score = linear_regression_cost.score(X_train, y_train)
stats_text = f'R² (test) = {r2_score:.4f}\n'
stats_text += f'RMSE = {np.sqrt(mse):.2f} mln'

plt.text(0.02, 0.95, stats_text,
         transform=plt.gca().transAxes,
         bbox=dict(facecolor='white',
                   edgecolor='gray',
                   alpha=0.8,
                   boxstyle='round,pad=0.5'),
         fontsize=10)

# Dodanie adnotacji dla ważnych punktów, jeśli potrzebne
max_cost_idx = np.argmax(y_test)
plt.annotate(f'Maksimum: {y_test[max_cost_idx][0]:.0f} mln',
             xy=(X_test[max_cost_idx], y_test[max_cost_idx]),
             xytext=(-50, 15), textcoords='offset points',
             bbox=dict(facecolor='white', edgecolor='gray', alpha=0.7),
             arrowprops=dict(arrowstyle='->'))

# Dostosowanie układu i marginesów 
plt.tight_layout()

# Pokazanie wykresu
plt.show()

## Drugi wykres - sprawdzenie

In [ ]:
plt.figure(figsize=(10, 6))

# Przedział ufności dla danych uczących
plt.fill_between(X_train.reshape(-1),
                 (train_predictions - confidence_interval).reshape(-1),
                 (train_predictions + confidence_interval).reshape(-1),
                 color='yellow', label='Odchylenie standardowe prognozy')

# Przedział ufności dla danych testowych
plt.fill_between(X_test.reshape(-1),
                 (test_predictions - confidence_interval).reshape(-1),
                 (test_predictions + confidence_interval).reshape(-1),
                 color='yellow')

# Punkty rzeczywistych kosztów dla danych treningowych
plt.scatter(X_train, y_train, label="Koszty rzeczywiste (dane uczące)")

# Linia przewidywanych kosztów dla danych treningowych
plt.plot(X_train, train_predictions, label='Koszty przewidywane (dane uczące)', color='red')

# Punkty rzeczywistych kosztów dla danych testowych
plt.scatter(X_test, y_test, label="Koszty rzeczywiste (dane testowe)", color='green')

# Linia przewidywanych kosztów dla danych testowych
plt.plot(X_test, test_predictions, label='Koszty przewidywane (dane testowe)', color='orange')

# Etykiety osi
plt.ylabel('Koszty [mln]')
plt.xlabel('Liczba zatrudnień')

# Legenda
plt.legend()

# Wyświetlenie wykresu
plt.show()


# Model kosztów w zależności od Zatrudnienia i Przychodu

In [ ]:
Z_train = fb_stats_y_df['Employment'].to_numpy().reshape(-1, 1)
P_train = fb_stats_y_df['Income'].to_numpy().reshape(-1, 1)
ZP_train = np.column_stack((Z_train, P_train))
C_train = fb_stats_y_df['Costs'].to_numpy().reshape(-1, 1)

Z_test = fb_stats_y_test_df['Employment'].to_numpy().reshape(-1, 1)
P_test = fb_stats_y_test_df['Income'].to_numpy().reshape(-1, 1)
ZP_test = np.column_stack((Z_test, P_test))
C_test = fb_stats_y_test_df['Costs'].to_numpy().reshape(-1, 1)

In [ ]:
linear_reg_costs = LinearRegression()
linear_reg_costs.fit(ZP_train, C_train)

print(f'Cost R^2: {linear_reg_costs.score(ZP_train, C_train):}')

## Bez logarytmizacji widzimy znaczne skoki w MSE. Spróbujmy zlogarytmizować dane i zobaczyć, czy to coś zmieni

In [ ]:
# Generowanie prognoz dla danych treningowych
y_hat = linear_reg_costs.predict(ZP_train)

# Rzeczywiste wartości kosztów (y_true)
y_true = C_train

# Macierz X dla funkcji `calculate_model_stats` - zawiera dane treningowe ZP_train
# Ta funkcja sama doda kolumnę jedynek, więc ZP_train powinno mieć tylko zmienne objaśniające.
X = ZP_train

# Użycie funkcji calculate_model_stats
stats = calculate_model_stats(y_hat, y_true, X)

# Wyświetlenie wyników
print("Statystyki modelu:", stats)

train_metrics_3 = calculate_prediction_metrics(C_train, linear_reg_costs.predict(ZP_train))
test_metrics_3 = calculate_prediction_metrics(C_test, linear_reg_costs.predict(ZP_test))

print_metrics_with_specific_units(train_metrics_3, "Model wielowymiarowy kosztów (dane treningowe)", "mln $")
print_metrics_with_specific_units(test_metrics_3, "Model wielowymiarowy kosztów (dane testowe)", "mln $")

In [ ]:
# %matplotlib notebook

# Znacznie szersze zakresy dla surface plot -> paramtry: gestosc siatki(100), zakresy osi x i y
axis_x = np.linspace(-5000, 100000, 100)  # Zwiększony zakres dla liczby pracowników
axis_y = np.linspace(-5000, 150000, 100)  # Zwiększony zakres dla przychodów
X, Y = np.meshgrid(axis_x, axis_y)

# Przekształć siatkę punktów do formatu wymaganego przez model
grid = np.vstack([X.ravel(), Y.ravel()]).T
Z = linear_reg_costs.predict(grid).reshape(X.shape)

fig = plt.figure(facecolor='white', figsize=(10, 8))
ax = fig.add_subplot(projection='3d')
ax.set_facecolor('white')

# Wykreślenie danych treningowych i testowych
ax.scatter(Z_train, P_train, C_train, label='Koszty prawdziwe(dane treningowe)', color='blue')
ax.scatter(Z_test, P_test, C_test, label='Koszty prawdziwe(dane testowe)', color='green')

# Wykreślenie powierzchni predykcji
surf = ax.plot_surface(X, Y, Z, alpha=0.3, label='Predykcje kosztow', color='orange')
surf._edgecolors2d = surf._edgecolor3d
surf._facecolors2d = surf._facecolor3d

ax.legend()
ax.set_xlabel('Liczba pracownikow')
ax.set_ylabel('Przychody [mln]')
ax.set_zlabel('Koszty [mln]')
plt.show()

## Funkcje do porownywania wydajnosci / sprawnosci modeli

In [ ]:
def plot_models_comparison():

    models = {
        'Szereg czasowy': (train_metrics_0, test_metrics_0),
        'Przychody': (train_metrics_1, test_metrics_1),
        'Koszty': (train_metrics_2, test_metrics_2),
        'Wielowymiarowy': (train_metrics_3, test_metrics_3)
    }
    
    metrics = ['R2', 'MAPE']
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    for i, metric in enumerate(metrics):
        train_values = [m[0][metric] for m in models.values()]
        test_values = [m[1][metric] for m in models.values()]
        
        x = np.arange(len(models))
        width = 0.35
        
        axes[i].bar(x - width/2, train_values, width, label='Train')
        axes[i].bar(x + width/2, test_values, width, label='Test')
        
        axes[i].set_title(f'{metric} dla różnych modeli')
        axes[i].set_xticks(x)
        axes[i].set_xticklabels(models.keys(), rotation=45)
        axes[i].legend()
        
    plt.tight_layout()
    plt.show()
    
plot_models_comparison()

In [ ]:
def plot_models_comparison():
    models = {
        'Szereg czasowy': (train_metrics_0, test_metrics_0),
        'Przychody': (train_metrics_1, test_metrics_1),
        'Koszty': (train_metrics_2, test_metrics_2),
        'Wielowymiarowy': (train_metrics_3, test_metrics_3)
    }
    
    # Zmieniamy zestaw metryk - usuwamy R2, dodajemy inne istotne metryki
    metrics = ['MAPE', 'RMSE', 'MAE']
    
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    for i, metric in enumerate(metrics):
        train_values = [m[0][metric] for m in models.values()]
        test_values = [m[1][metric] for m in models.values()]
        
        x = np.arange(len(models))
        width = 0.35
        
        axes[i].bar(x - width/2, train_values, width, label='Train', color='skyblue')
        axes[i].bar(x + width/2, test_values, width, label='Test', color='lightgreen')
        
        axes[i].set_title(f'{metric} dla różnych modeli')
        axes[i].set_xticks(x)
        axes[i].set_xticklabels(models.keys(), rotation=45)
        axes[i].legend()
        
        # Dodanie etykiet z wartościami
        for idx, value in enumerate(train_values):
            axes[i].text(idx - width/2, value, f'{value:.2f}', 
                        ha='center', va='bottom')
        for idx, value in enumerate(test_values):
            axes[i].text(idx + width/2, value, f'{value:.2f}', 
                        ha='center', va='bottom')
    
    plt.suptitle('Porównanie metryk oceny jakości predykcji dla różnych modeli', 
                 fontsize=14, y=1.05)
    
    # Dodanie opisów metryk
    fig.text(0.32, -0.1, 'MAPE - Średni błąd procentowy\n(mniejszy = lepszy)', 
             ha='center', va='center')
    fig.text(0.52, -0.1, 'RMSE - Pierwiastek błędu średniokwadratowego\n(mniejszy = lepszy)', 
             ha='center', va='center')
    fig.text(0.72, -0.1, 'MAE - Średni błąd bezwzględny\n(mniejszy = lepszy)', 
             ha='center', va='center')
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_models_comparison()

In [ ]:
def print_models_comparison_table():
    models = {
        'Szereg czasowy': (train_metrics_0, test_metrics_0),
        'Przychody': (train_metrics_1, test_metrics_1),
        'Koszty': (train_metrics_2, test_metrics_2),
        'Wielowymiarowy': (train_metrics_3, test_metrics_3)
    }
    
    print("\nPorównanie metryk dla wszystkich modeli:")
    print("-" * 100)
    print(f"{'Model':<15} {'Dane':<8} {'MAPE':>10} {'RMSE':>15} {'MAE':>15}")
    print("-" * 100)
    
    for model_name, (train_metrics, test_metrics) in models.items():
        # Dane treningowe
        print(f"{model_name:<15} {'Train':<8} {train_metrics['MAPE']:10.2f} "
              f"{train_metrics['RMSE']:15.2f} {train_metrics['MAE']:15.2f}")
        # Dane testowe
        print(f"{model_name:<15} {'Test':<8} {test_metrics['MAPE']:10.2f} "
              f"{test_metrics['RMSE']:15.2f} {test_metrics['MAE']:15.2f}")
        print("-" * 100)

In [ ]:
print_models_comparison_table()


## Redemption naszego modelu z 2 zmiennymi zaleznymi z logarytmizacja

In [ ]:
# 1. Model z logarytmizacją wszystkich zmiennych
# %matplotlib notebook
# Przygotowanie danych
Z_train_log = np.log10(Z_train)  # logarytm zatrudnienia
P_train_log = np.log10(P_train)  # logarytm przychodów
C_train_log = np.log10(C_train)  # logarytm kosztów
ZP_train_log = np.column_stack((Z_train_log, P_train_log))

Z_test_log = np.log10(Z_test)
P_test_log = np.log10(P_test)
C_test_log = np.log10(C_test)
ZP_test_log = np.column_stack((Z_test_log, P_test_log))

# Trenowanie modelu na zlogarytmowanych danych
linear_reg_costs_log = LinearRegression()

linear_reg_costs_log.fit(ZP_train_log, C_train_log)

# Ocena modelu
train_metrics_log = calculate_prediction_metrics(C_train_log, linear_reg_costs_log.predict(ZP_train_log))
test_metrics_log = calculate_prediction_metrics(C_test_log, linear_reg_costs_log.predict(ZP_test_log))

print_metrics_with_specific_units(train_metrics_log, "Model kosztów (log) - trenowanie", "log10(mln $)")
print_metrics_with_specific_units(test_metrics_log, "Model kosztów (log) - test", "log10(mln $)")

# Wizualizacja 3D dla modelu logarytmicznego
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Dane rzeczywiste
scatter1 = ax.scatter(Z_train_log, P_train_log, C_train_log, 
                     c='blue', label='Treningowe')
scatter2 = ax.scatter(Z_test_log, P_test_log, C_test_log, 
                     c='red', label='Testowe')

# Powierzchnia predykcji
x_range = np.linspace(Z_train_log.min(), Z_test_log.max(), 100)
y_range = np.linspace(P_train_log.min(), P_test_log.max(), 100)
X, Y = np.meshgrid(x_range, y_range)
XY = np.column_stack((X.ravel(), Y.ravel()))
Z = linear_reg_costs_log.predict(XY).reshape(X.shape)

surf = ax.plot_surface(X, Y, Z, alpha=0.3, cmap='viridis')

ax.set_xlabel('log10(Zatrudnienie)')
ax.set_ylabel('log10(Przychody)')
ax.set_zlabel('log10(Koszty)')
ax.legend()
plt.title('Model kosztów w skali logarytmicznej')
plt.show()

# Retrieve the coefficients and intercept from the trained model
a1 = linear_reg_costs_log.coef_[0][0]
a2 = linear_reg_costs_log.coef_[0][1]
b = linear_reg_costs_log.intercept_[0]

# Print the full equation of the model
print(f"Model H = {a1} * log10(Z) + {a2} * log10(P) + {b}")

In [ ]:
from sklearn.preprocessing import StandardScaler

# 2. Model z normalizacją danych
# Przygotowanie danych
scaler_Z = StandardScaler()
scaler_P = StandardScaler()
scaler_C = StandardScaler()

Z_train_norm = scaler_Z.fit_transform(Z_train)
P_train_norm = scaler_P.fit_transform(P_train)
C_train_norm = scaler_C.fit_transform(C_train)
ZP_train_norm = np.column_stack((Z_train_norm, P_train_norm))

Z_test_norm = scaler_Z.transform(Z_test)
P_test_norm = scaler_P.transform(P_test)
C_test_norm = scaler_C.transform(C_test)
ZP_test_norm = np.column_stack((Z_test_norm, P_test_norm))

# Trenowanie modelu na znormalizowanych danych
linear_reg_costs_norm = LinearRegression()
linear_reg_costs_norm.fit(ZP_train_norm, C_train_norm)

# Ocena modelu
train_metrics_norm = calculate_prediction_metrics(C_train_norm, 
                                               linear_reg_costs_norm.predict(ZP_train_norm))
test_metrics_norm = calculate_prediction_metrics(C_test_norm, 
                                              linear_reg_costs_norm.predict(ZP_test_norm))

print_metrics_with_specific_units(train_metrics_norm, 
                                "Model kosztów (normalized) - trenowanie", 
                                "znormalizowane jednostki")
print_metrics_with_specific_units(test_metrics_norm, 
                                "Model kosztów (normalized) - test", 
                                "znormalizowane jednostki")

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# 3. Model z cechami wielomianowymi
# Przygotowanie danych
poly = PolynomialFeatures(degree=2, include_bias=False)
ZP_train_poly = poly.fit_transform(ZP_train_norm)  # używamy znormalizowanych danych
ZP_test_poly = poly.transform(ZP_test_norm)

# Trenowanie modelu
linear_reg_costs_poly = LinearRegression()
linear_reg_costs_poly.fit(ZP_train_poly, C_train_norm)

# Ocena modelu
train_metrics_poly = calculate_prediction_metrics(C_train_norm, 
                                               linear_reg_costs_poly.predict(ZP_train_poly))
test_metrics_poly = calculate_prediction_metrics(C_test_norm, 
                                              linear_reg_costs_poly.predict(ZP_test_poly))

print_metrics_with_specific_units(train_metrics_poly, 
                                "Model kosztów (polynomial) - trenowanie", 
                                "znormalizowane jednostki")
print_metrics_with_specific_units(test_metrics_poly, 
                                "Model kosztów (polynomial) - test", 
                                "znormalizowane jednostki")

# linear_reg_costs_poly.coef_?? 
# ?linear_reg_costs_poly.coef_
# ??linear_reg_costs_poly.coef